In [148]:
import pandas as pd 
import streamlit as st
import functions as fc
import home
import sgs
import plotly.graph_objects as go

In [149]:
# Lista de códigos da API SGS BACEN
cod_list = pd.read_csv('base_csv/cod_ipca.csv', sep=';').set_index('codigo')
cod_list

,serie,lista
codigo,,
1635,Alimentos e bebidas,1635-Alimentos e bebidas
1636,Habitação,1636-Habitação
1637,Bens domésticos,1637-Bens domésticos
1638,Vestuário,1638-Vestuário
1639,Transportes,1639-Transportes
1640,Comunicação,1640-Comunicação
1641,Saúde e cuidados pessoais,1641-Saúde e cuidados pessoais
1642,Despesas pessoais,1642-Despesas pessoais
1643,Educação,1643-Educação


In [150]:
def df_sgs(lista_sgs,ano_inicio, ano_fim, name):
    df = pd.DataFrame()
    for i in lista_sgs:
        data_inicio = ano_inicio
        data_fim = ano_fim
        split = i.split(sep='-')
        df1 = sgs.time_serie(split[0], data_inicio, data_fim,True)
        df1 = pd.DataFrame(df1)
        df1 = df1.rename(columns={f'{split[0]}':f'{split[1]}'})
        df = pd.concat([df,df1], axis=1)
    return df.to_csv(f'base_csv/base_{name}.csv')

In [151]:
df_sgs(cod_list['lista'], '01/01/1980', '01/01/2021', 'ipca')

In [168]:
df = pd.read_csv('base_csv/base_ipca.csv', encoding='UTF-8', sep=',', index_col=0)
df1 = pd.DataFrame(df['IPCA Acumulado 12 meses'])
df1

,IPCA Acumulado 12 meses
2019-01-01,3.78
2019-02-01,3.89
2019-03-01,4.58
2019-04-01,4.94
2019-05-01,4.66
2019-06-01,3.37
2019-07-01,3.22
2019-08-01,3.43
2019-09-01,2.89
2019-10-01,2.54


In [174]:
def graf_plotly(data_frame,ano_inicio, ano_fim, titulo):
    fig = go.Figure()
    fig.update_layout(
    title= f'{titulo}',      
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        title= '%',
        showgrid=False,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white',
    legend = dict(title='Legenda')
    )
    for i in data_frame.columns:
        lines = fig.add_trace(go.Scatter(x=df.index, y=df[f'{i}'], name= f"{i}"  ))
    return lines

In [175]:
graf_plotly(df,f'01/01/1980', f'01/01/2021', 'IPCA - % Variação Mensal Total')